In [1]:
import numpy as np

In [2]:
class Matrix:
    def __init__(self, size):
        ## Array is defined by array[row][column]
        columns = size
        rows = size
        self.array = [[0 for _ in range(columns)] for _ in range(rows)]
        self.size = size
    
    def set_matrix(self, new_matrix):
        self.array = new_matrix
    
    def get_matrix(self):
        return self.array
    def set_value(self, row, column, value):
        self.array[row][column] = value
    
    def multiply(self, another_matrix, operator):
        other_array = another_matrix.get_matrix()
        assert len(other_array) == len(other_array[0]) == self.size 
        result = [[0 for _ in range(self.size)] for _ in range(self.size)]
        for thisRow in range(len(self.array)):
            for thisColumn in range(len(self.array[0])):
                result[thisRow][thisColumn] = self.__helper_mult(thisRow, thisColumn, other_array, operator)
        return result
    
    def __helper_mult(self, row, column, other_array, operator):
        total = operator.OP1_ID
        for c in range(len(other_array)):
            other_value = other_array[c][column]
            this_value = self.array[row][c]
            total = operator.op1(total,operator.op2(this_value, other_value))
        return total
    def copy(self):
        new_array = [row[:] for row in self.array]
        result = Matrix(size)
        result.set_matrix(new_array)
        return result
    
    def printSelf(self):
        print(np.matrix(self.array))
    
class AddMult:
    OP1_ID = 0
    OP2_ID = 1
    @staticmethod
    def op1(a, b):
        return a + b
    @staticmethod
    def op2(a, b):
        return a * b
    
def printMatrix(array):
    print(np.matrix(array))

In [4]:
m = Matrix(3)
m.set_value(0,0,1)
m.set_value(1,1,3)
m.set_value(2,2,1)
m.set_value(2,1,2)
m.printSelf()
print()
printMatrix(m.multiply(m, AddMult()))


[[1 0 0]
 [0 3 0]
 [0 2 1]]

[[1 0 0]
 [0 9 0]
 [0 8 1]]


In [48]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse




# We are going to create a class called LinkParser that inherits some
# methods from HTMLParser which is why it is passed into the definition
class LinkParser(HTMLParser):

    # This is a function that HTMLParser normally has
    # but we are adding some functionality to it
    def handle_starttag(self, tag, attrs):
        # We are looking for the begining of a link. Links normally look
        # like <a href="www.someurl.com"></a>
        if tag == 'a':
            for (key, value) in attrs:
                
                if key == 'href':
                    # We are grabbing the new URL. We are also adding the
                    # base URL to it. For example:
                    # www.netinstructions.com is the base and
                    # somepage.html is the new URL (a relative URL)
                    #
                    # We combine a relative URL with the base URL to create
                    # an absolute URL like:
                    # www.netinstructions.com/somepage.html
                    newUrl = parse.urljoin(self.baseUrl, value)
                    # And add it to our colection of links:
                    self.links = self.links + [newUrl]

    # This is a new function that we are creating to get links
    # that our spider() function will call
    def getLinks(self, url):
        self.links = []
        # Remember the base URL which will be important when creating
        # absolute URLs
        self.baseUrl = url
        # Use the urlopen function from the standard Python 3 library
        response = urlopen(url)
        # Make sure that we are looking at HTML and not other things that
        # are floating around on the internet (such as
        # JavaScript files, CSS, or .PDFs for example)
        if 'text/html' in response.getheader('Content-Type'):
            htmlBytes = response.read()
            # Note that feed() handles Strings well, but not bytes
            # (A change from Python 2.x to Python 3.x)
            htmlString = htmlBytes.decode("utf-8")
            self.feed(htmlString)
            
            
            return htmlString, self.links
        else:
            return "",[]
        

# And finally here is our spider. It takes in an URL, a word to find,
# and the number of pages to search through before giving up
def spider(url, word, maxPages):  
    pagesToVisit = [url]
    numberVisited = 0
    foundWord = False
    # The main loop. Create a LinkParser and get all the links on the page.
    # Also search the page for the word or string
    # In our getLinks function we return the web page
    # (this is useful for searching for the word)
    # and we return a set of links from that web page
    # (this is useful for where to go next)
    while numberVisited < maxPages and pagesToVisit != [] and not foundWord:
        numberVisited = numberVisited +1
        # Start from the beginning of our collection of pages to visit:
        url = pagesToVisit[0]
        pagesToVisit = pagesToVisit[1:]
        try:
            print(numberVisited, "Visiting:", url)
            parser = LinkParser()
            data, links = parser.getLinks(url)
            pagesToVisit = pagesToVisit + links

            if data.find(word)>-1:
                foundWord = True
                # Add the pages that we visited to the end of our collection
                # of pages to visit:
                print(" **Success!**")
        except:
            print(" **Failed!**")
    if foundWord:
        print("The word", word, "was found at", url)
    else:
        print("Word never found")
        
        
spider("https://www.hendrix.edu/", "Mark Goadrich", 40)


1 Visiting: https://www.hendrix.edu/
2 Visiting: https://www.hendrix.edu/life/
3 Visiting: https://www.hendrix.edu/academics/
4 Visiting: https://www.hendrix.edu/visit/
5 Visiting: https://www.hendrix.edu/financialaid/
6 Visiting: https://www.hendrix.edu/admission/
7 Visiting: https://www.hendrix.edu/apply
8 Visiting: https://www.hendrix.edu/currentstudents/
9 Visiting: https://www.hendrix.edu/parents/
10 Visiting: https://www.hendrix.edu/alumni/
11 Visiting: https://www.hendrix.edu/facultystaff/
12 Visiting: https://www.hendrix.edu/
13 Visiting: javascript:__doPostBack('ctl00$GlobalNav2015$LinkButton2','')
 **Failed!**
14 Visiting: https://www.hendrix.edu/CampusWeb
15 Visiting: https://outlook.office365.com/
16 Visiting: https://alumni.hendrix.edu/sslpage.aspx?pid=186
17 Visiting: https://www.hendrix.edu/news
18 Visiting: https://www.hendrix.edu/giving
19 Visiting: https://www.hendrix.edu/
20 Visiting: https://www.hendrix.edu/currentstudents/
21 Visiting: https://www.hendrix.edu/paren